# Importeer librarys

In [34]:
# Importeer gym dingen
import gym
from gym import Env
from gym.spaces import Discrete, Box, Tuple, Dict, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random 
import os

# Import stable baselines dingen
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.envs import SimpleMultiObsEnv

# Typer van spaces

In [58]:
Discrete(3).sample()

1

In [66]:
doos = Box(0, 0, shape = (2, 2), dtype = np.int64).sample()
doos[t[0]][t[1]] = 1
doos

array([[0, 1],
       [0, 0]])

In [55]:
Box(0, 1, shape = (2, 2)).sample()

array([[0.91917086, 0.4375294 ],
       [0.5371949 , 0.5679899 ]], dtype=float32)

In [4]:
Tuple((Discrete(3), Box(0,1, shape = (3,)))).sample()

(0, array([0.31743062, 0.22827569, 0.01403176], dtype=float32))

In [64]:
t = Tuple((Discrete(2), Discrete(2))).sample()
t

(0, 1)

In [5]:
Dict({'height':Discrete(2), 'speed':Box(0,100, shape = (1,))}).sample()

OrderedDict([('height', 0), ('speed', array([1.1749239], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [110]:
MultiDiscrete([2,2]).sample()

array([0, 1])

In [32]:
Box(low = 0, high = 100, shape = (1,)).sample()

array([66.4793], dtype=float32)

# Env voor containers

In [121]:
class Containers(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete([2,2])
        self.observation_space = Box(0, 0, shape = (2, 2), dtype = np.int64)
        self.state = [[0, 0],
                      [0, 0]]
        self.game_length = 4
    
    def step(self, action):
        
        klaar = False
        
        self.game_length -= 1
        
        if self.state[action[0]][action[1]] != 1:
            self.state[action[0]][action[1]] = 1
            reward = 1
        else:
            self.state[action[0]][action[1]] = 1
            reward = -1
            
        if self.game_length <= 0:
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.game_length = 4
        return self.observation_space
    
    

    

In [122]:
env = Containers()

In [101]:
t = env.action_space.sample()

int

In [99]:
env.observation_space.sample()

array([[0, 0],
       [0, 0]])

# Envorinment maken
- Build an agent to give us the best shower possible
- Randomly temparature
- 37 tot 39 degrees

In [8]:
class ShowerEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low = 0, high = 100, shape = (1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        # Verander de temperatuur
        self.state += action-1
        
        # Update de resterende douchesessie duur
        self.shower_length -= 1
        
        # Bereken de reward van de actie
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        # Controleer of de douchesessie is afgelopen
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        # Implement vizualisation
        pass
    
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
    

In [9]:
env = ShowerEnv()

In [10]:
env.observation_space.sample()

array([92.24891], dtype=float32)

In [11]:
env.action_space.sample()

2

# Test enviroment

In [123]:
episodes = 20
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{}, Score:{}'.format(episode, score))
env.close()

Episode:1, Score:2
Episode:2, Score:-4
Episode:3, Score:-2
Episode:4, Score:-4
Episode:5, Score:-4
Episode:6, Score:-4
Episode:7, Score:-4
Episode:8, Score:-4
Episode:9, Score:-4
Episode:10, Score:-4
Episode:11, Score:-4
Episode:12, Score:-4
Episode:13, Score:-4
Episode:14, Score:-4
Episode:15, Score:-4
Episode:16, Score:-4
Episode:17, Score:-4
Episode:18, Score:-4
Episode:19, Score:-4
Episode:20, Score:-4


# Model trainen

In [124]:
log_path = os.path.join('Training', 'Logs')
model =PPO('MlpPolicy', env, verbose = 1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [125]:
model.learn(total_timesteps=20000)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Box'